# Scraping HealthGrades for References

In [1]:
!pip3 install lxml
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
#example for exactly one doctor

url = 'https://www.healthgrades.com/physician/dr-craig-ehrensing-yxw7q'
def get_reviews(url):
    '''
    Takes in a url to any specific doctor and returns all of their reviews as strs in a list
    '''
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    #use Beautiful Soup to prettify the content
    soup = BeautifulSoup(response.content)
    soup.prettify()
    
    gender = None
    #determine gender 
    headdiv = soup.findAll("div", {"class":"summary-row summary-row-top"})
    if headdiv:
        for tag in headdiv:  
            genderInfo = tag.findAll("span", {"data-qa-target":"ProviderDisplayGender"})
            genderStr = str(genderInfo)
            result = re.search('<span data-qa-target="ProviderDisplayGender"> <!-- -->(.*)<!-- --> </span>', genderStr)
            gender = result.group(1)
    else:
        headdiv = soup.findAll("div", {"class": "summary-designated-header-image-left"})
        for tag in headdiv:  
            genderInfo = tag.findAll("span", {"data-qa-target":"ProviderDisplayGender"})
            genderStr = str(genderInfo)
            result = re.search('<span data-qa-target="ProviderDisplayGender"> <!-- -->(.*)<!-- --> </span>', genderStr)
            gender = result.group(1)
            
    #retrieve all comments
    commentdivs = soup.findAll("div", {"class": "c-single-comment__comment"})
    commentdivs
    reviews = []
    for div in commentdivs:
        s = str(div)
        s = s[1:]
        start = s.find(">")+1
        end = s.find("<")
        substring = s[start:end]
        reviews.append(substring)
    return gender, reviews

In [3]:
get_reviews('https://www.healthgrades.com/dentist/dr-lauren-rivet-3qyd9')

AttributeError: 'NoneType' object has no attribute 'group'

In [4]:
url2 = 'https://www.healthgrades.com/physician/dr-paula-hall-ypy2q'
get_reviews(url2)

AttributeError: 'NoneType' object has no attribute 'group'

In [5]:
link = 'https://www.healthgrades.com/find-a-doctor/a-z/a-1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(link, headers=headers)
soup = BeautifulSoup(response.content)
soup.prettify()
lnks = soup.findAll("a")
docs = []
for i in range(len(lnks)):
    trial = re.findall(r"<a href=\"/physician+/+", str(lnks[i]))
    if trial == ['<a href="/physician/']:
        #print(i)
        #print(lnks[i])
        #print(re.findall(r'</?a(|\s+[^>]+)>',str(lnks[i])))
        cut = re.findall(r'</?a(|\s+[^>]+)>',str(lnks[i]))
        #print(cut[0][6:])
        #print(re.findall(r"<a href=(\"[a-z]+)", str(lnks[i])))
        docs.append(cut[0][7:-1])
docs

['/physician/dr-justin-a-de-boer-cjybz',
 '/physician/dr-ruben-a-6jx16p1158',
 '/physician/dr-enenge-abodjedi-25pd2',
 '/physician/dr-yousif-a-rahim-35982',
 '/physician/dr-melanie-aaberg-3vtmd',
 '/physician/dr-randal-aaberg-y5snb',
 '/physician/dr-thomas-aaberg-ynfm8',
 '/physician/dr-thomas-aaberg-xxkf9',
 '/physician/dr-afeera-aabida-xr46l',
 '/physician/dr-meghan-aabo-xylnfmj',
 '/physician/dr-stella-aaboe-xj54y',
 '/physician/dr-aazy-aaby-33pd2',
 '/physician/dr-douglas-aach-2r68l',
 '/physician/dr-james-aachen-winans-2s2r5',
 '/physician/dr-venkat-aachi-yhmh5',
 '/physician/dr-venkat-aachi-xymnppf',
 '/physician/dr-kirk-aadalen-y84qp',
 '/physician/dr-richard-aadalen-ybnh3',
 '/physician/dr-abdul-aadam-y3b4f',
 '/physician/dr-ahmed-aadil-xylh45p',
 '/physician/dr-timothy-aadland-25tsl',
 '/physician/dr-gregory-aaen-3fqqf',
 '/physician/dr-nancy-aaen-ydrx4',
 '/physician/dr-victor-aaen-yd7t5',
 '/physician/dr-bradley-aafedt-2tjgl',
 '/physician/dr-osama-aaflaq-3g3lk',
 '/physicia

In [6]:

revs = {}
for doc in docs[1:]:
    link = "https://www.healthgrades.com" + doc
    try:
        (gender, commentList) = get_reviews(link)
        for comment in commentList:
            revs[comment] = gender
    except:
        print('not working: ', link)
revs


not working:  https://www.healthgrades.com/physician/dr-ruben-a-6jx16p1158
not working:  https://www.healthgrades.com/physician/dr-enenge-abodjedi-25pd2
not working:  https://www.healthgrades.com/physician/dr-yousif-a-rahim-35982
not working:  https://www.healthgrades.com/physician/dr-melanie-aaberg-3vtmd
not working:  https://www.healthgrades.com/physician/dr-randal-aaberg-y5snb
not working:  https://www.healthgrades.com/physician/dr-thomas-aaberg-ynfm8
not working:  https://www.healthgrades.com/physician/dr-thomas-aaberg-xxkf9
not working:  https://www.healthgrades.com/physician/dr-afeera-aabida-xr46l
not working:  https://www.healthgrades.com/physician/dr-meghan-aabo-xylnfmj
not working:  https://www.healthgrades.com/physician/dr-stella-aaboe-xj54y
not working:  https://www.healthgrades.com/physician/dr-aazy-aaby-33pd2
not working:  https://www.healthgrades.com/physician/dr-douglas-aach-2r68l
not working:  https://www.healthgrades.com/physician/dr-james-aachen-winans-2s2r5
not worki

not working:  https://www.healthgrades.com/physician/dr-daniel-aaron-26ws2
not working:  https://www.healthgrades.com/physician/dr-daniel-aaron-3bcgk
not working:  https://www.healthgrades.com/physician/dr-demetria-aaron-gdbxf
not working:  https://www.healthgrades.com/physician/dr-denise-aaron-y6syw
not working:  https://www.healthgrades.com/physician/dr-don-aaron-ydfvy
not working:  https://www.healthgrades.com/physician/dr-emily-aaron-xyllsjb
not working:  https://www.healthgrades.com/physician/dr-frank-aaron-xg7y2
not working:  https://www.healthgrades.com/physician/dr-geoffrey-aaron-gdxhp
not working:  https://www.healthgrades.com/physician/dr-gila-aaron-3fhd3
not working:  https://www.healthgrades.com/physician/dr-harry-aaron-32f9f
not working:  https://www.healthgrades.com/physician/dr-hayden-aaron-xymr2l9
not working:  https://www.healthgrades.com/physician/dr-hayden-aaron-yrqjh
not working:  https://www.healthgrades.com/physician/dr-houston-aaron-3d2vl
not working:  https://ww

{}

In [65]:
reviewsByGender = pd.DataFrame(list(revs.items()),columns = ['Review','Gender'])
reviewsByGender

,Review,Gender
0,"Exelent he really care about you, he is very p...",Male
1,Dr. A-Rahim was very knowledgeable about my lo...,Male
2,Horrible physician treats you like a kid. Pts ...,Female
3,Dr. Aaberg has been treating my macular degene...,Male
4,I am privileged to have Dr Aaberg in charge of...,Male
...,...,...
318,Didn’t find my cancer till it was too late. Be...,Male
319,Dr. Aaronson operated on me and oerforated my ...,Male
320,I feel very lucky to have Dr. Aaronson to take...,Male
321,"Dr. Aaronson can be a real jerk, if it wasn't ...",Male


# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Dr. Mattei, Tulane University. SimpleText Notebook. 2020. 
Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

In [86]:
# Vectorize the whole thing...
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Vectorize and play with token sizes...
vec = TfidfVectorizer(min_df = 0.05, 
                      max_df = 0.95, 
                      ngram_range=(1,1)) # play with min_df and max_df

reviews = reviewsByGender['Review']

# transform this into a sparse vector!
vec.fit(reviews)
tf_idf_sparse = vec.transform(reviews)
tf_idf_sparse


<323x1124 sparse matrix of type '<class 'numpy.float64'>'
	with 11279 stored elements in Compressed Sparse Row format>

In [ ]:
#get list of cleaned, stemmed adjectives, for later analysis. 
vect = CountVectorizer()
vect.fit(revs)
words = vect.get_feature_names()

adjectives = []
stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

In [51]:
print(tf_idf_sparse[0, :])

  (0, 1119)	0.25750188879510694
  (0, 1054)	0.19755180648853018
  (0, 971)	0.4691643202153929
  (0, 794)	0.3381696400563504
  (0, 769)	0.3279456279846566
  (0, 529)	0.17176840597097848
  (0, 460)	0.3346848731539662
  (0, 304)	0.13683432961799813
  (0, 206)	0.3580425548198684
  (0, 168)	0.27837594515512815
  (0, 77)	0.11951899251044311
  (0, 40)	0.2691272012325054


In [52]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    reviewsByGender['Gender'], 
                                                    test_size=0.2)

In [53]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

/Users/laurensussman/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [54]:
logisticRegr.classes_

array(['Female', 'Male'], dtype=object)

In [55]:
logisticRegr.coef_[:,0]

array([-0.30370039])

In [56]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [88]:
df_weights.sort_values('weight', ascending=False)[:25]

,word,weight
0,he,2.708874
76,his,1.633008
75,him,1.312151
12,was,0.581570
354,pain,0.574456
288,surgery,0.518147
8,and,0.515034
1,really,0.489406
119,best,0.478055
162,on,0.471679


In [93]:
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
119,best,0.478055
352,good,0.342027
583,overall,0.328955
309,friendly,0.302511
363,busy,0.268810
316,surgical,0.268187
461,efficient,0.253754
261,son,0.248755
1024,sleep,0.229133
204,confident,0.220970


In [92]:
df_weights.sort_values('weight', ascending=True)[:25]

,word,weight
343,her,-3.267136
41,she,-3.254604
506,lot,-0.748166
514,pregnancy,-0.706282
403,visit,-0.627232
83,patient,-0.624659
515,happy,-0.588430
97,always,-0.586132
762,listen,-0.527019
69,not,-0.523890


In [94]:
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
femaleAdjectives

,word,weight
515,happy,-0.588430
29,horrible,-0.507079
263,medical,-0.451521
851,attitude,-0.430110
209,excellent,-0.428584
509,whole,-0.409570
337,bed,-0.391738
828,aanestad,-0.390278
502,smart,-0.385961
516,lady,-0.330758


In [104]:
#verify results
agentic_terms = ['assertive','confident','aggressive','ambitious','dominan','forceful','independent','daring','outspoken','intellectua','earn','gain','know','insight','think']
communal_terms = ['sympathetic','kind','help','affection','sensitive','nurtur','agreeab','tactful','interpersonal','warm','car','tactful']
socio_communal = ['husband','wife','kid','babies','brother','child','colleague','family']

print('Results for most commonly found female adjectives:')
print('Agentic: ', femaleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', femaleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', femaleAdjectives['word'].isin(socio_communal).sum())
print('')
print('Results for most commonly found male adjectives:')
print('Agentic: ', maleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', maleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', maleAdjectives['word'].isin(socio_communal).sum())

Results for most commonly found female adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0

Results for most commonly found male adjectives:
Agentic:  1
Communal:  0
Socio-communal:  0
